In [ ]:
# 이미지 있는지 확인 (함수 필요하다면)

def check_img_tag(html):
  soup = BeautifulSoup(html, 'html.parser')
  img_tags = soup.find_all('img')
  if img_tags:
      return True
  else:
      return False

In [ ]:
# 있으면 이미지 함수 호출
# 이미지 함수
# 이미지 추출 후 일단 날짜로 이미지 이름을 받아오게 해둬서 date를 인자로 받게 했습니다
# 날짜인 이유: 파일명으로 쓸 수 없는 기호가 제목에 있으면 에러나고 끊겨서
# 병합시 추가 todo : 첫 목록에서 클릭해 들어갈 때 이미지 없는 경우 예외처리, csv column에 이미지 여부 확인하는 열 추가 후 저장여부 확인해주기

def get_and_save_img(date):
    # 이미지 추출
    find_img = driver.find_element(By.CLASS_NAME, 'view-con')
    img = find_img.find_elements(By.TAG_NAME, 'img')

    # 이미지가 없는 경우 None 반환
    if not img:
        return None

    img_src = img[0].get_attribute('src')

    # 이미지 파일 이름 생성
    filename = f'{directory}/{date}.jpg'
    count = 1
    while os.path.exists(filename):
        filename = f'{directory}/{date}_{count}.jpg'
        count += 1

    # 이미지 다운로드
    response = requests.get(img_src)

    # 이미지를 로컬 파일로 저장
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"{date} 이미지를 성공적으로 저장했습니다.")
    else:
        print(f"{date} 이미지를 다운로드할 수 없습니다.")

    return img_src  # 이미지가 있는 경우 이미지 소스 반환

In [ ]:
# 텍스트 함수

## 글내용 스크랩 함수
def get_content():
    # 페이지의 소스 가져오기
    html_source = driver.page_source

    # BeautifulSoup 객체 생성하여 HTML 파싱
    soup = BeautifulSoup(html_source, 'html.parser')

    # CSS 선택자를 사용하여 내용 추출
    content = soup.select_one("div.view-con").get_text(strip=False) if soup.select_one("div.view-con") else ''

    # 텍스트 정리
    content = content.replace("\t", "").replace('\xa0', '')

    # 글내용 반환
    return content


## 제목, 날짜 스크랩 함수.
def get_titleDate():
    # 페이지의 소스 가져오기
    html_source = driver.page_source

    # BeautifulSoup 객체 생성하여 HTML 파싱
    soup = BeautifulSoup(html_source, 'html.parser')

    # CSS 선택자를 사용하여 날짜, 제목 추출
    title = soup.select_one("div.board-view-info div.view-info h2").get_text(strip=True) if soup.select_one("div.board-view-info div.view-info h2") else ''
    date = soup.select_one("div.view-info div.view-detail dl.write dd").get_text(strip=True) if soup.select_one("div.view-info div.view-detail dl.write dd") else ''

    # 텍스트 정리
    title = title.replace("\t", "").replace('\xa0', '')

    # 제목, 날짜 반환 : ('제목', '날짜') 튜플 형태
    return title, date

In [ ]:
# 웹 드라이버 및 url 설정
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import os
import requests

In [ ]:
# Chrome WebDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

# Chrome WebDriver 초기화 및 웹 페이지 열기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

# 학사공지 홈페이지 접속
# url = "https://www.hufs.ac.kr/hufs/11282/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGaHVmcyUyRjIxODElMkZhcnRjbExpc3QuZG8lM0Y%3D"

# 장학공지 홈페이지 접속
url = "https://www.hufs.ac.kr/hufs/11283/subview.do"
driver.get(url)

# csv에 담아 출력할 DataFrame 생성
import pandas as pd
df_crawling = pd.DataFrame(columns = ['index', 'date', 'title', 'content'])

# 이미지 다운로드 디렉토리 생성 -> 폴더 이름 수정 원할 시 directory명 변경
directory = 'test_final'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# index 수집
# Selenium으로 페이지의 HTML 소스 가져오기
html = driver.page_source

# BeautifulSoup 객체 생성하여 HTML 파싱
soup = BeautifulSoup(html, 'html.parser')

# CSS 선택자로 인덱스 추출
elements = soup.select("div._fnctWrap table tbody tr:not(.notice) td.td-num")
index_texts = [element.get_text() for element in elements]

# 추출한 인덱스 형변환
index = int(index_texts[0]) if index_texts else None

In [ ]:
# 첫번째 공지 글 클릭
# CSS 선택자를 사용하여 공지태그가 아닌 첫 번째 글의 <a> 태그를 찾기
first_page = driver.find_element(By.CSS_SELECTOR, "div.scroll-table tbody tr:not(.notice) a")

# 클릭
first_page.click()

In [ ]:
while True:
    try:
        title = get_titleDate()[0]
        date = get_titleDate()[1]
        img_flag = check_img_tag(html)
        if img_flag:
            get_and_save_img(date)
        content = get_content()

        # 잘 추출되었는지 확인
        print(f"Index: {index}")
        print(f"Date: {date}")
        print(f"Title: {title}")
        print(f"Content: {content}")

        # 데이터프레임에 추가
        new_row = {'index': index, 'date': date, 'title': title, 'content': content}
        df_crawling.loc[len(df_crawling)] = new_row

        # 인덱스 갱신
        index -= 1

        # 이전글 클릭
        # CSS 선택자를 사용하여 이전글의 <a> 태그를 찾기
        previous_page = driver.find_element(By.CSS_SELECTOR, "main.contents div._fnctWrap div.view-navi dl:first-child dd a")

        # 클릭
        previous_page.click()
        time.sleep(2)

    except Exception as e:
        print(e)
        break

In [ ]:
# Selenium WebDriver 종료
driver.quit()

In [ ]:
# csv 파일 export
import os
df_crawling.to_csv(os.path.join(os.path.expanduser('~'), 'Desktop', 'noti_janghak.csv'), encoding="UTF-8", index=False)

# csv 형식
- 컬럼: 인덱스, 제목, 날짜, 내용
  - 만약 내용에 이미지만 있다면
    - 로컬에 이미지 저장하기
    - csv 파일에는 '이미지만 존재' 출력하기 (추후 텍스트 추출 후 원활한 csv 파일 통합 위함)